In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [2]:
# importing stocks, using static list to avoid paying for api (for now)
stocks = pd.read_csv('sp_500_stocks.csv')

In [3]:
# acquire API token
# sandbox mode from IED cloud
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

# for each stock: need market capitalization and price
# first API call
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
price = data['latestPrice']
market_cap = data['marketCap']

print(price)
print(market_cap)

124.41
2133769187530


In [4]:
# add stocks data to pd dataframe
cols = ['Ticker', 'Stock Price', 'Market Cap', '# shares to buy']

# final_dataframe = pd.DataFrame(columns = cols)

# for symbol in stocks['Ticker']:
#     api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
#     data = requests.get(api_url).json()
#     final_dataframe = final_dataframe.append(
#                                         pd.Series([symbol, 
#                                                    data['latestPrice'], 
#                                                    data['marketCap'], 
#                                                    'N/A'], 
#                                                   index = cols), 
#                                         ignore_index = True)
# final_dataframe

In [5]:
# Batch API calls   

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

final_dataframe = pd.DataFrame(columns = cols)



In [6]:
for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = cols), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Stock Price,Market Cap,# shares to buy
0,A,117.38,35811811878,N/A
1,AAL,20.57,13379929714,N/A
2,AAP,177.38,11559980703,N/A
3,AAPL,122.50,2113563555237,N/A
4,ABBV,107.40,192734659017,N/A
...,...,...,...,...
500,YUM,105.28,32092812062,N/A
501,ZBH,160.40,33074844982,N/A
502,ZBRA,479.76,26138008073,N/A
503,ZION,57.32,9404787177,N/A


In [7]:
portfolio_size = input("Enter the value of your portfolio:")
try:
    val = float(portfolio_size)
except ValueError:
    print("Not a number. run this again")

Enter the value of your portfolio:1000000


In [8]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, '# shares to buy'] = math.floor(position_size / final_dataframe['Stock Price'][i])
final_dataframe

,Ticker,Stock Price,Market Cap,# shares to buy
0,A,117.38,35811811878,16
1,AAL,20.57,13379929714,96
2,AAP,177.38,11559980703,11
3,AAPL,122.50,2113563555237,16
4,ABBV,107.40,192734659017,18
...,...,...,...,...
500,YUM,105.28,32092812062,18
501,ZBH,160.40,33074844982,12
502,ZBRA,479.76,26138008073,4
503,ZION,57.32,9404787177,34


In [10]:
# Format excel output
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [11]:
# String format for tickers
# \$XX.XX format for stock prices
# \$XX,XXX format for market capitalization
# Integer format for the number of shares to purchase

background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [12]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [13]:
writer.save()